In [0]:
%pip install beautifulsoup4

In [0]:
dbutils.library.restartPython()

In [0]:
dbutils.widgets.text("wn_catalog", "dbdemos")
dbutils.widgets.text("wn_schema", "lookup")
dbutils.widgets.text("wn_table", "dbr_version")

In [0]:
# Get widget values
wn_catalog = dbutils.widgets.get("wn_catalog")
wn_schema = dbutils.widgets.get("wn_schema")
wn_table = dbutils.widgets.get("wn_table")

In [0]:
from delta.tables import DeltaTable
from bs4 import BeautifulSoup
from pyspark.sql.functions import to_date, coalesce, regexp_extract, when, col, try_to_timestamp, lit
import requests

In [0]:
# Check if the catalog already exists. More elaborate way of checking to prevent error if max catalogs have already been created
existing_catalogs = spark.sql(f"SHOW CATALOGS LIKE '{wn_catalog}'")
if existing_catalogs.count() == 0:
    create_catalog_sql = f"CREATE CATALOG IF NOT EXISTS {wn_catalog}"
    spark.sql(create_catalog_sql)

In [0]:
create_schema_sql = f"CREATE SCHEMA IF NOT EXISTS {wn_catalog}.{wn_schema}"
spark.sql(create_schema_sql)

In [0]:
full_table_name = f"{wn_catalog}.{wn_schema}.{wn_table}"
print(full_table_name)

In [0]:
# Step 1: Get the page content
url = 'https://learn.microsoft.com/en-us/azure/databricks/release-notes/runtime/'

# Send a GET request to the webpage
response = requests.get(url)
response.raise_for_status()

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find the specific section by heading ID
target_heading = soup.find('h2', {'id': 'all-supported-databricks-runtime-releases'})

# Initialize variables for table extraction
extracted_table = None

if target_heading:
    # Find the next table sibling after the target heading
    target_table = target_heading.find_next('table')
    
    if target_table:
        # Clean and normalize headers for Delta Lake compatibility
        headers = [
            th.text.strip()
            .lower()  # Convert to lowercase
            .replace(' ', '_')  # Replace spaces
            .replace('-', '_')  # Replace hyphens
            for th in target_table.find_all('th')
        ]
        
        # Extract table rows
        rows = []
        for row in target_table.find_all('tr'):
            row_data = [td.text.strip() for td in row.find_all('td')]
            if row_data:  # Skip header row
                rows.append(row_data)
        
        # Create result structure
        extracted_table = {
            'headers': headers,
            'data': rows
        }

display(extracted_table)


In [0]:
df = (
    spark.createDataFrame(extracted_table['data'], extracted_table['headers'])
    .withColumnRenamed('version', 'dbr_version')
    .withColumnRenamed('variants', 'dbr_variants')
    .withColumn('clean_dbr_version', regexp_extract(col('dbr_version'), r'\d+\.\d', 0))
    .withColumn(
        'release_date',
        coalesce(
            try_to_timestamp(col('release_date'), lit('MMM d, yyyy')).cast('date'),
            try_to_timestamp(col('release_date'), lit('MMMM d, yyyy')).cast('date')
        )
    )
    .withColumn(
        'end_of_support_date',
        coalesce(
            try_to_timestamp(col('end_of_support_date'), lit('MMM d, yyyy')).cast('date'),
            try_to_timestamp(col('end_of_support_date'), lit('MMMM d, yyyy')).cast('date')
        )
    )
    .withColumn('dbr_is_lts', when(col('dbr_version').contains('LTS'), True).otherwise(False))
)
display(df)

In [0]:
if not spark.catalog.tableExists(full_table_name):
    df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(full_table_name)
else:
    delta_table = DeltaTable.forName(spark, full_table_name)
    (delta_table.alias("target")
     .merge(
         df.alias("source"),
         "target.clean_dbr_version = source.clean_dbr_version"
     )
     .whenMatchedUpdateAll()  # Update existing records
     .whenNotMatchedInsertAll()  # Insert new records
     .execute()
    )

In [0]:
'''
# Create fake data for merging
fake_data = [
    # Row 1: Updates existing 16.2 record with new end-of-support date
    ["16.2", "Databricks Runtime 16.2Databricks Runtime 16.2 for Machine Learning", 
     "3.5.0", "Feb 5, 2025", "Oct 5, 2025", "16.2", False],
     
    # Row 2: Inserts a completely new version (17.0)
    ["17.0", "Databricks Runtime 17.0Databricks Runtime 17.0 for Machine Learning", 
     "3.4.0", "Sep 15, 2024", "Mar 15, 2026", "17.0", False]
]

# Define schema matching your original table
columns = ["dbr_version", "dbr_variants", "apache_spark_version", 
          "release_date", "end_of_support_date", "clean_dbr_version", "dbr_is_lts"]

# Create DataFrame with the fake data
df = (spark.createDataFrame(fake_data, columns).withColumn('release_date', coalesce(to_date('release_date', 'MMM d, yyyy'), to_date('release_date', 'MMMM d, yyyy')))
    .withColumn('end_of_support_date', coalesce(to_date('end_of_support_date', 'MMM d, yyyy'), to_date('end_of_support_date', 'MMMM d, yyyy')))
) 
display(df)
'''